In [1]:
# Import necessary libraries
from sklearn.linear_model import *
from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import  LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import *
from sklearn.compose import *
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.model_selection import *
from sklearn.ensemble import VotingClassifier, BaggingClassifier
from sklearn.discriminant_analysis import *
from sklearn.preprocessing import *
from sklearn.neighbors import *
from sklearn.svm import SVC
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.linear_model import *
import os

warnings.simplefilter('ignore')

os.chdir('D:\kaggla_comp\smoker')

train = pd.read_csv('train.csv')
train

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159251,159251,40,155,45,69.0,1.5,2.0,1,1,127,...,72,159,14.5,1,0.8,25,26,13,0,0
159252,159252,50,155,75,82.0,1.0,1.0,1,1,120,...,64,108,14.5,1,0.6,21,20,18,0,0
159253,159253,40,160,50,66.0,1.5,1.0,1,1,114,...,87,93,10.9,1,0.6,15,9,12,0,0
159254,159254,50,165,75,92.0,1.2,1.0,1,1,121,...,55,80,14.4,1,1.1,22,17,37,0,1


In [2]:
# Assign the training dataset without the 'smoking' column to 'x'
x = train.drop('smoking', axis=1)

# Assign the 'smoking' column from the training dataset to 'y'
y = train['smoking']

# Split the training data into training and testing sets, with a test size of 30% and stratified sampling based on the 'smoking' column
# This ensures that the class distribution in the training and testing sets is similar to the original dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=24)

# Create a Logistic Regression model
lr = LogisticRegression()

# Create a Bagging Classifier using the Logistic Regression model as the base estimator
# Set the number of estimators to 15 and enable out-of-bag (OOB) score calculation
bagg = BaggingClassifier(estimator=lr, n_estimators=15, random_state=24, oob_score=True)

# Fit the Bagging Classifier to the training data
bagg.fit(x_train, y_train)

# Print the out-of-bag (OOB) score, which is an estimate of the model's performance on unseen data
print("OOB Score:", bagg.oob_score_)

OOB Score: 0.7243068201185874


In [3]:
# Use the trained Bagging Classifier to predict the classes of the test data
y_pred = bagg.predict(x_test)

# Get the predicted probabilities of the positive class (1) for the test data
y_pred_prob = bagg.predict_proba(x_test)[:, 1]

# Calculate and print the accuracy score of the predictions
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Calculate and print the log loss of the predicted probabilities
print("Log Loss:", log_loss(y_test, y_pred_prob))

Accuracy Score: 0.7194256650689662
Log Loss: 0.5457670511752435


In [4]:
# Print the parameters of the Bagging Classifier
print(bagg.get_params())

{'base_estimator': 'deprecated', 'bootstrap': True, 'bootstrap_features': False, 'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__l1_ratio': None, 'estimator__max_iter': 100, 'estimator__multi_class': 'auto', 'estimator__n_jobs': None, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'lbfgs', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(), 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 15, 'n_jobs': None, 'oob_score': True, 'random_state': 24, 'verbose': 0, 'warm_start': False}


# tunning

In [5]:
# Create a Stratified K-Fold cross-validation object with 5 splits, shuffling the data, and a random state of 24
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

# Create a Bagging Classifier with Logistic Regression as the base estimator
# Set the number of estimators to 15 and enable out-of-bag (OOB) score calculation
bagg = BaggingClassifier(estimator=lr, n_estimators=15, random_state=24, oob_score=True)

# Fit the Bagging Classifier to the training data
bagg.fit(x_train, y_train)

# Define the parameter grid for the Logistic Regression model within the Bagging Classifier
params = {
    'estimator__penalty': ['L2', None],
    'estimator__C': np.linspace(0.001, 10, 5),
    'estimator__solver': ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag']
}

# Create a GridSearchCV object to perform cross-validation and hyperparameter tuning
# Use the Bagging Classifier as the estimator, the parameter grid defined above, the Stratified K-Fold cross-validation
# Set the scoring metric to 'roc_auc' (Area Under the Receiver Operating Characteristic Curve)
gcv = GridSearchCV(bagg, param_grid=params, cv=kfold, verbose=3, scoring='roc_auc')

# Fit the GridSearchCV object to the entire dataset (x and y)
gcv.fit(x, y)

# Print the best parameters found by the GridSearchCV
print("Best Parameters:", gcv.best_params_)

# Print the best score (ROC AUC) found by the GridSearchCV
print("Best Score:", gcv.best_score_)

# multipe estimator

In [ ]:
# Create a Gaussian Naive Bayes classifier
nb = GaussianNB()

# Create a Stratified K-Fold cross-validation object with 5 splits, shuffling the data, and a random state of 24
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

# Create a Support Vector Classifier (SVC) with probability estimates enabled and a random state of 24
svm = SVC(random_state=24, probability=True)

# Create a Decision Tree Classifier with a random state of 24
dtc = DecisionTreeClassifier(random_state=24)

# Create a Logistic Regression model
lr = LogisticRegression()

# Create a Bagging Classifier with a random state of 24
bagg = BaggingClassifier(random_state=24)

# Define the parameter grid for the Bagging Classifier
params = {
    'estimator': [svm, nb, lr, dtc],
    'n_estimators': [10, 15]
}

# Create a GridSearchCV object to perform cross-validation and hyperparameter tuning
# Use the Bagging Classifier as the estimator, the parameter grid defined above, the Stratified K-Fold cross-validation,
# set the verbosity to 3, and use the negative log loss as the scoring metric
gcv_m = GridSearchCV(bagg, param_grid=params, cv=kfold, verbose=3, scoring='neg_log_loss')

# Fit the GridSearchCV object to the entire dataset (x and y)
gcv_m.fit(x, y)

# Print the best parameters found by the GridSearchCV
print("Best Parameters:", gcv_m.best_params_)

# Print the best score (negative log loss) found by the GridSearchCV
print("Best Score:", gcv_m.best_score_)

# inferencing

In [ ]:
best_model = gcv.best_estimator_

test_data = pd.read_csv("test.csv",index_col=0)

pred_berry = best_model.predict(test_data)

submit = pd.read_csv("sample_submission.csv")

submit['smoking'] = pred_berry

submit.to_csv("D:\kaggla_comp\smoker\bootstrap_smok_25.csv", index=False)

# run this code later it takes to much time to execute